In [ ]:
import random
import matplotlib.pyplot as plt

class MemoryBlock:
    """
    Represents a block of memory with a starting address and size.

    Attributes:
        start (int): The starting address of the memory block.
        size (int): The size of the memory block.
    """
    def __init__(self, start: int, size: int):
        self.start = start
        self.size = size

    def __repr__(self) -> str:
        return f"[Start: {self.start}, Size: {self.size}]"


class MemoryAllocator:
    """
    Simulates a memory allocator using different allocation strategies.
    Supports allocation, deallocation, memory compaction, and visualization.

    Attributes:
        total_size (int): Total size of memory available.
        free_blocks (list): List of free memory blocks.
        allocated_blocks (dict): Dictionary of allocated memory blocks by process ID.
    """
    def __init__(self, total_size: int):
        """
        Initializes memory with a single free block of the given size.

        Args:
            total_size (int): Total size of the memory.
        """
        self.total_size = total_size
        self.free_blocks = [MemoryBlock(0, total_size)]
        self.allocated_blocks = {}

    def allocate(self, pid: str, size: int, strategy: str = 'first_fit') -> None:
        """
        Allocates memory for a process using a specified strategy.

        Args:
            pid (str): Process ID.
            size (int): Amount of memory to allocate.
            strategy (str): Allocation strategy ('first_fit', 'best_fit', or 'worst_fit').
        """
        block_index = -1

        if strategy == 'first_fit':
            for i, block in enumerate(self.free_blocks):
                if block.size >= size:
                    block_index = i
                    break

        elif strategy == 'best_fit':
            min_size = float('inf')
            for i, block in enumerate(self.free_blocks):
                if block.size >= size and block.size < min_size:
                    min_size = block.size
                    block_index = i

        elif strategy == 'worst_fit':
            max_size = -1
            for i, block in enumerate(self.free_blocks):
                if block.size >= size and block.size > max_size:
                    max_size = block.size
                    block_index = i

        if block_index == -1:
            print(f"Allocation failed for process {pid}: Not enough memory.")
            return

        block = self.free_blocks[block_index]
        allocated_block = MemoryBlock(block.start, size)
        self.allocated_blocks[pid] = allocated_block

        block.start += size
        block.size -= size
        if block.size == 0:
            self.free_blocks.pop(block_index)

        print(f"Process {pid} allocated at address {allocated_block.start} with size {size}.")

    def deallocate(self, pid: str) -> None:
        """
        Deallocates memory assigned to a process.

        Args:
            pid (str): Process ID to deallocate.
        """
        if pid not in self.allocated_blocks:
            print(f"Deallocation failed: Process {pid} not found.")
            return

        block = self.allocated_blocks.pop(pid)
        self.free_blocks.append(block)
        self.free_blocks = sorted(self.free_blocks, key=lambda b: b.start)
        self.merge_free_blocks()

        print(f"Process {pid} deallocated from address {block.start}.")

    def merge_free_blocks(self) -> None:
        """
        Merges adjacent free memory blocks to reduce fragmentation.
        """
        merged = []
        for block in self.free_blocks:
            if not merged:
                merged.append(block)
            else:
                last = merged[-1]
                if last.start + last.size == block.start:
                    last.size += block.size
                else:
                    merged.append(block)
        self.free_blocks = merged

    def compact(self) -> None:
        """
        Moves all allocated memory blocks to the start and consolidates free space.
        """
        if not self.allocated_blocks:
            self.free_blocks = [MemoryBlock(0, self.total_size)]
            print("Memory compacted. All memory is free.")
            return

        sorted_allocs = sorted(self.allocated_blocks.items(), key=lambda item: item[1].start)
        current_address = 0
        for pid, block in sorted_allocs:
            block_size = block.size
            self.allocated_blocks[pid] = MemoryBlock(current_address, block_size)
            current_address += block_size

        total_allocated = current_address
        self.free_blocks = [MemoryBlock(total_allocated, self.total_size - total_allocated)]
        print("Memory compacted. All allocated blocks moved to the start.")

    def display(self) -> None:
        """
        Prints the current state of free and allocated memory blocks.
        """
        print("\nFree Memory Blocks:")
        for block in self.free_blocks:
            print(block)

        print("\nAllocated Memory Blocks:")
        for pid, block in self.allocated_blocks.items():
            print(f"Process {pid}: {block}")

    def visualize(self) -> None:
        """
        Displays a bar chart representing the memory layout.
        """
        fig, ax = plt.subplots(figsize=(10, 2))
        ax.set_xlim(0, self.total_size)
        ax.set_ylim(0, 1)
        ax.set_title("Memory Layout")

        for block in self.free_blocks:
            ax.barh(0.5, block.size, left=block.start, height=0.5, color='green', edgecolor='black', label='Free')

        for pid, block in self.allocated_blocks.items():
            ax.barh(0.5, block.size, left=block.start, height=0.5, label=f'Allocated: {pid}', edgecolor='black')

        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1.05, 1))
        plt.tight_layout()
        plt.show()


def simulate_random_allocations(allocator: MemoryAllocator, n: int) -> None:
    """
    Simulates 'n' random memory allocations with random strategies and sizes.

    Args:
        allocator (MemoryAllocator): The memory allocator instance.
        n (int): Number of allocations to simulate.
    """
    for i in range(n):
        pid = f"P{i+1}"
        size = random.randint(5, 30)
        strategy = random.choice(['first_fit', 'best_fit', 'worst_fit'])
        allocator.allocate(pid, size, strategy)


def command_line_interface() -> None:
    """
    Command-line interface to interact with the memory allocator.
    """
    allocator = MemoryAllocator(100)
    while True:
        print("\nOptions:")
        print("1. Allocate Memory")
        print("2. Deallocate Memory")
        print("3. Display Memory")
        print("4. Visualize Memory")
        print("5. Simulate Random Allocations")
        print("6. Compact Memory")
        print("7. Exit")
        choice = input("Choose an option: ")

        if choice == '1':
            pid = input("Enter process ID: ")
            size = int(input("Enter size to allocate: "))
            strategy = input("Enter strategy (first_fit / best_fit / worst_fit): ")
            allocator.allocate(pid, size, strategy)
        elif choice == '2':
            pid = input("Enter process ID to deallocate: ")
            allocator.deallocate(pid)
        elif choice == '3':
            allocator.display()
        elif choice == '4':
            allocator.visualize()
        elif choice == '5':
            n = int(input("How many random allocations? "))
            simulate_random_allocations(allocator, n)
        elif choice == '6':
            allocator.compact()
        elif choice == '7':
            break
        else:
            print("Invalid option.")


if __name__ == "__main__":
    command_line_interface()



Options:
1. Allocate Memory
2. Deallocate Memory
3. Display Memory
4. Visualize Memory
5. Simulate Random Allocations
6. Compact Memory
7. Exit
